In [ ]:
import numpy as np
import simulation

# Simulation parameters
N = 5  # Number of particles
timesteps = 20000  # Number of simulation time steps
dt = 1e-2  # Timestep
alpha = 0.1  # cubic coupling
beta = 0.1  # quartic coupling

qs, ps = simulation.integration(N, timesteps, dt, alpha, beta)

# Import simulation
# qs = np.load("D:\School\Magistrska\data\\" + "_" + str(N) + "_" + str(timesteps) + "_" + str(dt) + "_" + str(alpha) + "_" + str(beta) + ".npy")

In [ ]:
qs_mean = np.mean(qs)
qs_std = np.std(qs)

def preprocess(X):
  X = (X - qs_mean) / qs_std
  return X

def postprocess(X):
  X = X * qs_std + qs_mean
  return X

qs_mean, qs_std

In [ ]:
np.arange(timesteps), np.sin(np.arange(timesteps))

In [ ]:
preprocess(qs)

In [ ]:
def qs_to_X_y(data, window_size=5):
  X = []
  y = []
  
  for i in range(len(data)-window_size):
    X.append(data[i: i + window_size])
    y.append(data[i + window_size])
    
  return np.array(X), np.array(y)

In [ ]:
window_size = 1000
X, y = qs_to_X_y(qs, window_size)
X.shape, y.shape

In [ ]:
train_size = 0.7
val_size = 0.2
test_size = 0.1

tr=int(timesteps*train_size)
va=int(timesteps*val_size)
te=int(timesteps*test_size)

X_train, y_train = X[:tr], y[:tr]
X_val, y_val = X[-va:], y[-va:]
X_test, y_test = X[tr: tr +  te], y[tr: tr + te]

# LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
tf.keras.backend.set_floatx('float64')

model_lstm = Sequential()
model_lstm.add(InputLayer((window_size, N)))
# model_lstm.add(LSTM(64, return_sequences=True, activation= "tanh"))
model_lstm.add(LSTM(128, activation="tanh"))
# model_lstm.add(Dense(8, 'relu'))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(N, 'linear'))

model_lstm.summary()

In [ ]:
cp = ModelCheckpoint('model_lstm/', save_best_only=True)
model_lstm.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])

In [ ]:
epochs = 10
batchsize = 250

model_lstm.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batchsize, callbacks=[cp], verbose = 2, workers=-2, use_multiprocessing=True)

In [ ]:
from tensorflow.keras.models import load_model
model_lstm = load_model('model_lstm/')

In [ ]:
test_pred_lstm = np.copy(X_test[0])

for k in range(len(y_test)-window_size):
    pred = model_lstm.predict(test_pred_lstm[np.newaxis, -window_size:])
    test_pred_lstm = np.concatenate((test_pred_lstm, pred), axis = 0)
    
test_pred_lstm = postprocess(test_pred_lstm)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

# Create the data for the 3D plot
x = np.arange(N)
y = np.arange(te)
x, y = np.meshgrid(x, y)
z = postprocess(qs[tr: tr + te])-test_pred_lstm

# Create a figure and axes
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Plot the 3D surface
ax.plot_surface(x, y, z, cmap='plasma')

# Set labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title("lstm")

# Rotate the plot
ax.view_init(elev=40, azim=65)

# Show the plot
plt.show()


In [ ]:
for particles in range(N):
    plt.plot(np.arange(tr, tr+te), postprocess(qs[tr: tr + te, particles]), color="tab:blue")
    plt.plot(np.arange(tr, tr+te), test_pred_lstm[:, particles], color="tab:orange")

    # Set labels and title
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title("lstm")
    plt.ylim(-1, 1)

    # Show the plot
    plt.show()

# CNN

In [ ]:
model_cnn = Sequential()
model_cnn.add(InputLayer((window_size, N)))
model_cnn.add(Conv1D(64, kernel_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(8, 'relu'))
model_cnn.add(Dense(N, 'linear'))

model_cnn.summary()

In [ ]:
cp2 = ModelCheckpoint('model_cnn/', save_best_only=True)
model_cnn.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
epochs=20
batchsize=16

# model_cnn.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batchsize, callbacks=[cp2], verbose = 2, use_multiprocessing=True)

In [ ]:
test_pred_cnn = np.copy(X_test[0])

for k in range(len(y_test)-window_size):
    pred = model_cnn.predict(test_pred_cnn[np.newaxis, -window_size:])
    test_pred_cnn = np.concatenate((test_pred_cnn, pred), axis = 0)
    
test_pred_cnn = postprocess(test_pred_cnn)

In [ ]:
%matplotlib inline

# Create the data for the 3D plot
x = np.arange(N)
y = np.arange(te)
X, Y = np.meshgrid(x, y)
Z = postprocess(qs[tr: tr + te]) - test_pred_cnn

# Create a figure and axes
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Plot the 3D surface
ax.plot_surface(X, Y, Z, cmap='plasma')

# Set labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title("cnn")

# Rotate the plot
ax.view_init(elev=40, azim=65)

# Show the plot
plt.show()


In [ ]:
for particles in range(N):
    plt.plot(range(te), postprocess(qs[tr: tr + te, particles]))
    plt.plot(range(te), test_pred_cnn[:, particles])

    # Set labels and title
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title("lstm")

    # Show the plot
    plt.show()

# GRU

In [ ]:
model_gru = Sequential()
model_gru.add(InputLayer((window_size, N)))
model_gru.add(GRU(64))
model_gru.add(Dense(8, 'relu'))
model_gru.add(Dense(N, 'linear'))
model_gru.summary()

In [ ]:
cp3 = ModelCheckpoint('model_gru/', save_best_only=True)
model_gru.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
epochs=20
batchsize=16

model_gru.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batchsize, callbacks=[cp3], verbose = 2, workers=7, use_multiprocessing=True)

In [ ]:
test_pred_gru = np.copy(X_test[0])

for k in range(len(y_test)-window_size):
    pred = model_gru.predict(test_pred_gru[np.newaxis, -window_size:])
    test_pred_gru = np.concatenate((test_pred_gru, pred), axis = 0)
    
test_pred_gru = postprocess(test_pred_gru)

In [ ]:
%matplotlib inline

# Create the data for the 3D plot
x = np.arange(N)
y = np.arange(te)
X, Y = np.meshgrid(x, y)
Z = postprocess(qs[tr: tr + te]) - test_pred_gru

# Create a figure and axes
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Plot the 3D surface
ax.plot_surface(X, Y, Z, cmap='plasma')

# Set labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title("gru")

# Rotate the plot
ax.view_init(elev=40, azim=65)

# Show the plot
plt.show()


In [ ]:
for particles in range(N):
    plt.plot(range(te), postprocess(qs[tr: tr + te, particles]))
    plt.plot(range(te), test_pred_gru[:, particles])

    # Set labels and title
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title("lstm")

    # Show the plot
    plt.show()
